## Import Libraries

In [ ]:
import numpy as np, random, operator, pandas as pd, matplotlib.pyplot as plt

## Import Data

In [ ]:
df_codata = pd.read_csv("dantzig42.csv", header=None, names=['X', 'Y'])
df_codata

## Define Classes

### City Class

In [ ]:
class City:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def distance(self, city):
        xDis = abs(self.x - city.x)
        yDis = abs(self.y - city.y)
        distance = np.sqrt((xDis ** 2) + (yDis ** 2))
        return distance
    
    def __repr__(self):
        return "(" + str(self.x) + "," + str(self.y) + ")"

### Fitness Class

In [ ]:
class Fitness:
    def __init__(self, route):
        self.route = route
        self.distance = 0
        self.fitness= 0.0
    
    def routeDistance(self):
        if self.distance ==0:
            pathDistance = 0
            for i in range(0, len(self.route)):
                fromCity = self.route[i]
                toCity = None
                if i + 1 < len(self.route):
                    toCity = self.route[i + 1]
                else:
                    toCity = self.route[0]
                pathDistance += fromCity.distance(toCity)
            self.distance = pathDistance
        return self.distance
    
    def routeFitness(self):
        if self.fitness == 0:
            self.fitness = 1 / float(self.routeDistance())
        return self.fitness

## Initialize Population

In [ ]:
def createRoute(cityList):
    route = random.sample(cityList, len(cityList))
    return route

In [ ]:
def initialPopulation(popSize, cityList):
    population = []

    for i in range(0, popSize):
        population.append(createRoute(cityList))
    return population

## Rank 

In [ ]:
def rankRoutes(population):
    fitnessResults = {}
    for i in range(0,len(population)):
        fitnessResults[i] = Fitness(population[i]).routeFitness()
    return sorted(fitnessResults.items(), key = operator.itemgetter(1), reverse = True)

## Selection

In [ ]:
def selection(popRanked, eliteSize):
    selectionResults = []
    df = pd.DataFrame(np.array(popRanked), columns=["Index","Fitness"])
    df['cum_sum'] = df.Fitness.cumsum()
    df['cum_perc'] = 100*df.cum_sum/df.Fitness.sum()
    
    for i in range(0, eliteSize):
        selectionResults.append(popRanked[i][0])
    for i in range(0, len(popRanked) - eliteSize):
        pick = 100*random.random()
        for i in range(0, len(popRanked)):
            if pick <= df.iat[i,3]:
                selectionResults.append(popRanked[i][0])
                break
    return selectionResults

## Crossover
#### Choose Intended 'Breed' Function From the Following:

### Ordinal Crossover

In [ ]:
def breed(parent1, parent2):
    child = [None] * len(parent1)
    geneA = int(random.random() * len(parent1))
    geneB = int(random.random() * len(parent1))

    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)

    for i in range(startGene, endGene):
        child[i] = parent1[i]

    not_added = [item for item in parent2 if item not in child]
    idx = 0
    for i in range(len(child)):
        if child[i] is None:
            child[i] = not_added[idx]
            idx += 1

    return child

### PMX

In [ ]:
def breed(parent1, parent2):
    child = [None] * len(parent1)
    geneA = int(random.random() * len(parent1))
    geneB = int(random.random() * len(parent1))

    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)

    for i in range(startGene, endGene):
        child[i] = parent1[i]

    for i in range(startGene, endGene):
        if parent2[i] not in child:
            idx = parent2.index(parent1[i])
            while child[idx] is not None:
                idx = parent2.index(parent1[idx])
            child[idx] = parent2[i]

    for i in range(len(child)):
        if child[i] is None:
            child[i] = parent2[i]

    return child

### Ordered Crossover

In [ ]:
def breed (parent1, parent2):
    child = [None] * len (parent1)
    geneA = int(random. random () * len (parent1))
    geneB = int (random. random () * len (parent1))
    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)
    
    for i in range(startGene, endGene):
        child [i] = parent1 [i]
    
    remaining = [city for city in parent2 if city not in child]
    index = 0
    
    for i in range (len (parent2)):
        if child[i] is None:
            child[i] = remaining [index]
            index += 1
    return child

### Edge Recombination

In [ ]:
def breed(parent1, parent2):
    child = []
    cities = len(parent1)
    edges_p1 = {parent1[i]: [parent1[(i + 1) % cities], parent1[(i - 1) % cities]] for i in range(cities)}
    edges_p2 = {parent2[i]: [parent2[(i + 1) % cities], parent2[(i - 1) % cities]] for i in range(cities)}

    # Convert keys to City objects in the dictionaries
    edges_p1 = {City(city.x, city.y): [City(adjacent[0].x, adjacent[0].y), City(adjacent[1].x, adjacent[1].y)] for city, adjacent in edges_p1.items()}
    edges_p2 = {City(city.x, city.y): [City(adjacent[0].x, adjacent[0].y), City(adjacent[1].x, adjacent[1].y)] for city, adjacent in edges_p2.items()}

    start_city = parent1[0]  # Start from the first city in the parent1

    while len(child) < cities:
        child.append(start_city)

        neighbors_p1 = edges_p1.get(start_city, [])
        neighbors_p2 = edges_p2.get(start_city, [])

        for neighbor in neighbors_p1:
            if start_city in edges_p1.get(neighbor, []):
                edges_p1[neighbor] = [city for city in edges_p1[neighbor] if city != start_city]
        for neighbor in neighbors_p2:
            if start_city in edges_p2.get(neighbor, []):
                edges_p2[neighbor] = [city for city in edges_p2[neighbor] if city != start_city]

        if start_city in edges_p1:
            del edges_p1[start_city]
        if start_city in edges_p2:
            del edges_p2[start_city]

        if edges_p1 or edges_p2:
            next_cities = edges_p1.get(start_city, []) + edges_p2.get(start_city, [])
            counts = {city: next_cities.count(city) for city in next_cities}
            if counts:
                start_city = min(counts, key=counts.get)
            else:
                remaining_cities = [city for city in parent1 if city not in child]
                if remaining_cities:
                    start_city = remaining_cities[0]
                else:
                    break  # Break the loop if there are no remaining cities
        else:
            remaining_cities = [city for city in parent1 if city not in child]
            if remaining_cities:
                start_city = remaining_cities[0]
            else:
                break  # Break the loop if there are no remaining cities

    return child


### Crossover Function

In [ ]:
def matingPool(population, selectionResults):
    matingpool = []
    for i in range(0, len(selectionResults)):
        index = selectionResults[i]
        matingpool.append(population[index])
    return matingpool

In [ ]:
def breedPopulation(matingpool, eliteSize):
    children = []
    length = len(matingpool) - eliteSize
    pool = random.sample(matingpool, len(matingpool))

    for i in range(0,eliteSize):
        children.append(matingpool[i])
    
    for i in range(0, length):
        child = breed(pool[i], pool[len(matingpool)-i-1])
        children.append(child)
    return children

## Local Search

### 2-OPT

In [ ]:
def two_Opt(route, i, k):
    new_route = route[:i] + route[i:k+1][::-1] + route[k+1:]
    return new_route

### 3-OPT

In [ ]:
def three_Opt(route, i, j, k):
    # 3개의 하위 경로 
    part1 = route[:i]
    part2 = route[i:j]
    part3 = route[j:k]
    part4 = route[k:]

    # 3개의 하위 경로 합치는 조합
    option1 = part1 + part3 + part2 + part4
    option2 = part1 + part2 + part4 + part3
    option3 = part1 + part4 + part3 + part2

    # 각 옵션의 거리 계산
    dist_option1 = sum(city.distance(option1[idx + 1]) for idx, city in enumerate(option1[:-1]))
    dist_option2 = sum(city.distance(option2[idx + 1]) for idx, city in enumerate(option2[:-1]))
    dist_option3 = sum(city.distance(option3[idx + 1]) for idx, city in enumerate(option3[:-1]))

    # 최소 거리인 옵션 선택
    min_dist = min(dist_option1, dist_option2, dist_option3)
    if min_dist == dist_option1:
        new_route = option1
    elif min_dist == dist_option2:
        new_route = option2
    else:
        new_route = option3

    return new_route


### 5-OPT

In [ ]:
def five_Opt(route, i, j, k, l, m):
    # 5개의 하위 경로
    part1 = route[:i]
    part2 = route[i:j]
    part3 = route[j:k]
    part4 = route[k:l]
    part5 = route[l:m]
    part6 = route[m:]

    # 5개의 하위 경로를 합치는 조합
    option1 = part1 + part5 + part4 + part3 + part2 + part6
    option2 = part1 + part2 + part6 + part5 + part4 + part3
    option3 = part1 + part2 + part3 + part6 + part5 + part4
    option4 = part1 + part2 + part4 + part5 + part3 + part6
    option5 = part1 + part6 + part3 + part2 + part5 + part4

    # 각 옵션의 거리 계산
    dist_option1 = sum(city.distance(option1[idx + 1]) for idx, city in enumerate(option1[:-1]))
    dist_option2 = sum(city.distance(option2[idx + 1]) for idx, city in enumerate(option2[:-1]))
    dist_option3 = sum(city.distance(option3[idx + 1]) for idx, city in enumerate(option3[:-1]))
    dist_option4 = sum(city.distance(option4[idx + 1]) for idx, city in enumerate(option4[:-1]))
    dist_option5 = sum(city.distance(option5[idx + 1]) for idx, city in enumerate(option5[:-1]))

    # 최소 거리인 옵션 선택
    min_dist = min(dist_option1, dist_option2, dist_option3, dist_option4, dist_option5)
    if min_dist == dist_option1:
        new_route = option1
    elif min_dist == dist_option2:
        new_route = option2
    elif min_dist == dist_option3:
        new_route = option3
    elif min_dist == dist_option4:
        new_route = option4
    else:
        new_route = option5

    return new_route


## Mutation
#### Define Intended Local Search Algorithm Name in line 4 of Following Code (If None, Delete Line):

In [ ]:
def mutate(individual, mutationRate):
    if random.random() < mutationRate:
        i, j, k, l, m = sorted(random.sample(range(len(individual)), 5))
        individual = five_Opt(individual, i, j, k, l, m)
    return individual

In [ ]:
def mutatePopulation(population, mutationRate):
    mutatedPop = []
    
    for ind in range(0, len(population)):
        mutatedInd = mutate(population[ind], mutationRate)
        mutatedPop.append(mutatedInd)
    return mutatedPop


## Replacement

In [ ]:
def nextGeneration(currentGen, eliteSize, mutationRate):
    popRanked = rankRoutes(currentGen)
    selectionResults = selection(popRanked, eliteSize)
    matingpool = matingPool(currentGen, selectionResults)
    children = breedPopulation(matingpool, eliteSize)
    nextGeneration = mutatePopulation(children, mutationRate)
    return nextGeneration

## GA 

In [ ]:
def geneticAlgorithm(population, popSize, eliteSize, mutationRate, generations):
    pop = initialPopulation(popSize, population)
    print("Initial distance: " + str(1 / rankRoutes(pop)[0][1]))
    
    for i in range(0, generations):
        pop = nextGeneration(pop, eliteSize, mutationRate)
    
    print("Final distance: " + str(1 / rankRoutes(pop)[0][1]))
    bestRouteIndex = rankRoutes(pop)[0][0]
    bestRoute = pop[bestRouteIndex]
    return bestRoute

## Running the GA

In [ ]:
cityList = []

for i in range(len(df_codata)):
    X, Y = df_codata.values[i]
    cityList.append(City(x=int(X), y=int(Y)))

print(len(cityList))

In [ ]:
geneticAlgorithm(population=cityList, popSize=100, eliteSize=20, mutationRate=0.01, generations=500)

## Plot and Save Best Fitness

In [ ]:
def geneticAlgorithmPlot(population, popSize, eliteSize, mutationRate, generations):
    pop = initialPopulation(popSize, population)
    progress = []
    progress.append(1 / rankRoutes(pop)[0][1])
    
    for i in range(0, generations):
        pop = nextGeneration(pop, eliteSize, mutationRate)
        progress.append(1 / rankRoutes(pop)[0][1])
    
    plt.plot(progress)
    plt.ylabel('Distance')
    plt.xlabel('Generation')
    
    # Save the plot
    plt.savefig('PMXX-5OPT.png')

    # Display the plot
    plt.show()


In [ ]:
geneticAlgorithmPlot(population=cityList, popSize=100, eliteSize=20, mutationRate=0.01, generations=500)